In [445]:
import pandas as pd
from datetime import datetime
from slugify import slugify
from currency_converter import CurrencyConverter
import numpy as np

In [446]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [447]:
now = datetime.now()
dt_string = now.strftime("%Y-%m-%d %H:%M:%S")

In [448]:
df = pd.read_csv('csv/products.csv')
images = pd.read_csv('out/image.csv')
price = pd.read_csv('out/price.csv')

In [449]:
df['Price 1'] = df['Price 1'].str.replace(",", ".").astype(float)
df['Price 2'] = df['Price 2'].str.replace(",", ".").astype(float)
df['Price 3'] = df['Price 3'].str.replace(",", ".").astype(float)
df['Price 4'] = df['Price 4'].str.replace(",", ".").astype(float)
df['Price 5'] = df['Price 5'].str.replace(",", ".").astype(float)

In [450]:
df['Rating 1'] = df['Rating 1'].astype(float)
df['Rating 2'] = df['Rating 2'].astype(float)
df['Rating 3'] = df['Rating 3'].astype(float)
df['Rating 4'] = df['Rating 4'].astype(float)
df['Rating 5'] = df['Rating 5'].astype(float)

In [451]:
c = CurrencyConverter()
def convert_currency(x, y):
    try:
        return c.convert(float(x), str(y), 'USD')
    except:
        return np.nan

In [452]:
df['Price 1 USD'] = df.apply(lambda x: convert_currency(x['Price 1'], x['Curr1']), axis=1).astype(float).round(2)
df['Price 2 USD'] = df.apply(lambda x: convert_currency(x['Price 2'], x['Curr2']), axis=1).astype(float).round(2)
df['Price 3 USD'] = df.apply(lambda x: convert_currency(x['Price 3'], x['Curr3']), axis=1).astype(float).round(2)
df['Price 4 USD'] = df.apply(lambda x: convert_currency(x['Price 4'], x['Curr4']), axis=1).astype(float).round(2)
df['Price 5 USD'] = df.apply(lambda x: convert_currency(x['Price 5'], x['Curr5']), axis=1).astype(float).round(2)

In [453]:
tempDf = pd.DataFrame()

In [454]:
tempDf['minPrice'] = df[['Price 1 USD', 'Price 2 USD', 'Price 3 USD', 'Price 4 USD', 'Price 5 USD']]\
[(df['Price 1 USD'].notnull()) | \
 (df['Price 2 USD'].notnull()) | \
 (df['Price 3 USD'].notnull()) | \
 (df['Price 4 USD'].notnull()) | \
 (df['Price 5 USD'].notnull())
].min(axis=1).astype(float).round(2)

In [455]:
tempDf['maxPrice'] = df[['Price 1 USD', 'Price 2 USD', 'Price 3 USD', 'Price 4 USD', 'Price 5 USD']]\
[(df['Price 1 USD'].notnull()) | \
 (df['Price 2 USD'].notnull()) | \
 (df['Price 3 USD'].notnull()) | \
 (df['Price 4 USD'].notnull()) | \
 (df['Price 5 USD'].notnull())
].max(axis=1).astype(float).round(2)

In [456]:
tempDf['avgPrice'] = df[['Price 1 USD', 'Price 2 USD', 'Price 3 USD', 'Price 4 USD', 'Price 5 USD']]\
[(df['Price 1 USD'].notnull()) | \
 (df['Price 2 USD'].notnull()) | \
 (df['Price 3 USD'].notnull()) | \
 (df['Price 4 USD'].notnull()) | \
 (df['Price 5 USD'].notnull())
].mean(axis=1).astype(float).round(2)

In [457]:
df1 = tempDf[['minPrice']]
df2 = tempDf[['maxPrice']]
df3 = tempDf[['avgPrice']]

In [458]:
df1 = df1.rename(columns={'minPrice' : 'Price'})
df2 = df2.rename(columns={'maxPrice' : 'Price'})
df3 = df3.rename(columns={'avgPrice' : 'Price'})

In [459]:
newDf1 = pd.concat([df1, df2, df3]).dropna()

In [460]:
newDf1['id'] = [i for i in range(price.shape[0]+1, price.shape[0]+1 + newDf1.shape[0])]
newDf1['ID'] = [i for i in range(price.shape[0]+1, price.shape[0]+1 + newDf1.shape[0])]
newDf1.rename(columns = {'Price':'amount'}, inplace = True)
newDf1['currencyId'] = 'USD'
newDf1['amountMainCurrency'] = newDf1['amount']
newDf1['mainCurrencyId'] = 'USD'

In [461]:
price = pd.concat([price, newDf1])

In [462]:
df['avgRating'] = df[['Rating 1', 'Rating 2', 'Rating 3', 'Rating 4', 'Rating 5']]\
[(df['Rating 1'].notnull()) | \
 (df['Rating 2'].notnull()) | \
 (df['Rating 3'].notnull()) | \
 (df['Rating 4'].notnull()) | \
 (df['Rating 5'].notnull())
].mean(axis=1)

In [463]:
df['reviewCount'] = df[['Number Review 1', 'Number Review 2', 'Number Review 3', 'Number Review 4', 'Number Review 5']]\
[(df['Number Review 1'].notnull()) | \
 (df['Number Review 2'].notnull()) | \
 (df['Number Review 3'].notnull()) | \
 (df['Number Review 4'].notnull()) | \
 (df['Number Review 5'].notnull())
].count(axis=1).astype(int)

In [482]:
def get_price_id(value):
    return int(price[:][price['amount'] == value]['id'].values[0])

In [483]:
df['minPriceId'] = tempDf['minPrice'].apply(get_price_id).astype(int)
df['maxPriceId'] = tempDf['maxPrice'].apply(get_price_id).astype(int)
df['averagePriceId'] = tempDf['avgPrice'].apply(get_price_id).astype(int)

In [466]:
newDf = df[['Name', 'E-Tickets', 'Description', 'ID', 'avgRating', 'reviewCount', \
           'minPriceId', 'maxPriceId', 'averagePriceId']]

In [467]:
columns = ['createdDate', 'updatedDate', 'internalNote', 'creatorId', 'statusId', \
          'slug', 'shortDescription', 'alternativeName', 'mainImageId']

In [468]:
newDf = pd.concat([newDf, pd.DataFrame(columns = columns)])

In [469]:
newDf.rename(columns = {'Name':'name', 'E-Tickets' : 'onlineTicketUrl', \
                    'Description' : 'description', 'ID' : 'id', 'avgRating' : 'averageRating'}, inplace = True)

In [470]:
import random
import time
 
 
def str_time_prop(start, end, time_format, prop):
    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))
    ptime = stime + prop * (etime - stime)
    return time.strftime(time_format, time.localtime(ptime))
 
 
def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)
 

In [471]:
newDf['createdDate'] = [random_date("2021-1-1", "2022-4-30", random.random()) for i in range(1, newDf.shape[0] + 1)]
newDf['updatedDate'] = [random_date("2021-1-1", "2022-4-30", random.random()) for i in range(1, newDf.shape[0] + 1)]
newDf['internalNote'] = ''
newDf['statusId'] = 4
newDf['creatorId'] = 900
newDf['id'] = newDf.id.astype(int)

In [472]:
def set_slug(value):
    try:
        return slugify(str(value))
    except:
        return ''

newDf['slug'] = newDf['name'].apply(set_slug)

In [473]:
def get_short_description(value):
    return str(value)[:200]

newDf['shortDescription'] = newDf['description'].apply(get_short_description)

In [474]:
def get_alternative_name(value):
    temp = value.split(' ')
    return ', '.join(str(e) for e in temp)

newDf['alternativeName'] = newDf['name'].apply(get_alternative_name)

In [475]:
def get_main_image_id(value):
    try:
        tempName = str(value) + '_1'
        return images[:][images['name'].str.contains(tempName)]['id'].values[0]
    except:
        return ''
    
newDf['mainImageId'] = df['ID'].apply(get_main_image_id)

In [485]:
price.head()

,amount,currencyId,ID,id,amountMainCurrency,mainCurrencyId
0,350000,IDR,1,1,350000,IDR
1,25,USD,2,2,25,USD
2,350000,IDR,3,3,350000,IDR
3,450000,IDR,4,4,450000,IDR
4,843000,IDR,5,5,843000,IDR


In [481]:
newDf.to_csv('out/products.csv', index=False, encoding='utf-8')
price.to_csv('out/price.csv', index=False, encoding='utf-8')